In [ ]:
import numpy as np
import pandas as pd
import ast
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# =======================
# Import Libraries
# =======================
import numpy as np
import pandas as pd
import ast

# =======================
# Load Data
# =======================
movies = pd.read_csv(r'E:\JOB ASSESMENTS\tmdb_5000_movies.csv')
credits = pd.read_csv(r'E:\JOB ASSESMENTS\tmdb_5000_credits.csv')

# Merge datasets
movies = movies.merge(credits, on='title')

# Keep relevant columns
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]


In [4]:
# =======================
# Data Preprocessing
# =======================

def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

def convert_top3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter += 1
    return L

def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ", ""))
    return L1


In [7]:
# =======================
# Import Libraries
# =======================
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# =======================
# Load Data
# =======================
movies = pd.read_csv(r"E:\JOB ASSESMENTS\tmdb_5000_movies.csv")
credits = pd.read_csv(r"E:\JOB ASSESMENTS\tmdb_5000_credits.csv")

movies = movies.merge(credits, on='title')
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

# =======================
# Helper Functions
# =======================
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

def convert_top3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter += 1
    return L

def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ", ""))  # remove spaces for consistency
    return L1

# =======================
# Data Preprocessing
# =======================
movies.dropna(inplace=True)

movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert_top3)
movies['crew'] = movies['crew'].apply(fetch_director)

movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)

# Split overview into words
movies['overview'] = movies['overview'].apply(lambda x: x.split())

# Create "tags" column
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

# Keep only useful columns
new = movies[['movie_id', 'title', 'tags']]
new['tags'] = new['tags'].apply(lambda x: " ".join(x))

# =======================
# Feature Extraction
# =======================
cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new['tags']).toarray()

similarity = cosine_similarity(vector)

# =======================
# Recommendation Function
# =======================
def recommend(movie):
    if movie not in new['title'].values:
        print(f"❌ '{movie}' not found in dataset.")
        return
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    print(f"\n🎬 Movies similar to '{movie}':\n")
    for i in distances[1:6]:
        print("👉", new.iloc[i[0]].title)

# =======================
# Example Run
# =======================
recommend("Avatar")


ModuleNotFoundError: No module named 'sklearn'